# OpenAI API를 이용한 댓글 감정 분석

이 강의에서는 OpenAI의 Chat API를 활용하여 주어진 댓글의 감정을 자동으로 분석하는 프로그램을 작성해보겠습니다. 감정 분석(Sentiment Analysis)은 텍스트 데이터를 기반으로 해당 텍스트가 긍정적인지 부정적인지를 판단하는 자연어 처리(NLP)의 중요한 과제 중 하나입니다. 이 코드는 리뷰 데이터를 입력받아 이를 긍정 또는 부정으로 분류하게 됩니다.

# 🧠 GPT를 활용한 온라인 댓글 감정 분석: 큰 그림으로 이해하기

이 실습은 복잡한 코드보다, **"GPT를 이용해 댓글이 긍정적인지 부정적인지 구별하는 자동화 시스템"**을 만드는 데 목적이 있습니다. 아래 내용을 따라가면서 전체 구조를 하나의 이야기처럼 이해해봅시다.

---

## 🧩 1. 이 프로젝트, 무슨 일을 하나요?

우리는 수많은 **댓글(review)** 들을 모아, 그것들이 **긍정적인지(1)** 또는 **부정적인지(-1)** 자동으로 판단하게 만듭니다.

사람이 하나하나 보고 분류하지 않고, **OpenAI의 인공지능 GPT**가 그 역할을 대신합니다.

---

## 🔁 2. 전반적인 흐름: 한 줄 요약

**"댓글 하나 → GPT에게 감정 분석 요청 → 결과 비교 → 정확도 계산"**

이 과정을 수십~수백 개의 댓글에 대해 자동으로 반복합니다.

---

## 🧱 3. 구조를 큰 블록으로 나누면?

1. **데이터 불러오기**  
   - 댓글과 정답(긍정 or 부정)이 담긴 엑셀 파일을 불러옵니다.
   
2. **GPT API 설정하기**  
   - OpenAI의 GPT에 접근하기 위한 준비를 합니다 (열쇠=API Key).

3. **감정 분석 함수 정의**  
   - GPT에게 "이 댓글이 긍정이야, 부정이야?"라고 물어보는 역할을 합니다.

4. **댓글 하나하나 분석 시작**  
   - 각 댓글을 하나씩 GPT에게 보내고, 실제 정답과 비교합니다.

5. **정확도 계산**  
   - 전체 댓글 중 맞춘 개수의 비율을 계산해 성능을 측정합니다.

---

## 🎯 4. 왜 이렇게 구조를 짰을까?

- **사람 대신 GPT가 판단**하도록 함으로써 감정 분석의 자동화를 실현합니다.
- **정확도를 실시간으로 비교**해가며 성능을 검증할 수 있습니다.
- 여러 실험(모델 변경, 파라미터 변경, 프롬프트 조정 등)을 통해 **성능 개선**까지 체험합니다.

---

## ✨ 예시로 쉽게 이해해보기

> 🤖 GPT야, "이 제품 정말 좋아요, 최고예요!" 이거 긍정이야 부정이야?  
> → GPT: 1 (긍정)

> 🤖 GPT야, "배송도 늦고 제품도 불량이었어요."  
> → GPT: -1 (부정)

> 우리가 원하는 건 이런 걸 **수천 개 자동으로 처리하는 시스템**이에요.

---



## 1. 데이터 불러오기 및 기본 구조 설정

우선, pandas를 이용하여 리뷰 데이터를 불러오고, 데이터의 기본적인 구조를 확인합니다.


In [ ]:
!pip install openai --upgrade

In [ ]:
import pandas as pd

# 리뷰 데이터 불러오기 (reviews.xlsx 파일을 읽어서 데이터프레임으로 저장)
reviews = pd.read_excel('/content/total.xlsx')

# 데이터의 크기(행, 열) 출력
print(reviews.shape)

# 데이터의 첫 5줄 미리보기 (데이터 구조를 확인하는 용도)
reviews.head()


(2279, 8)


,Num,파일명,text,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중,label
0,1,1_전주팔복예술공장리뷰,전주에서 진보된 전시공간,0.0,3,0.7294,"[(0, 0.09138874), (1, 0.086825), (2, 0.0924325...",NaN
1,2,1_전주팔복예술공장리뷰,굿,1.0,0,0.2500,"[(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]",NaN
2,3,1_전주팔복예술공장리뷰,전주 진입해서 가까운 거리에 예술을 접할 수 있는 멋진곳 써니 감성의 카페도 좋고 ...,2.0,1,0.8997,"[(0, 0.033970088), (1, 0.899678), (2, 0.033449...",NaN
3,4,1_전주팔복예술공장리뷰,자연과 어우러져 아름다워요,3.0,0,0.2500,"[(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]",NaN
4,5,1_전주팔복예술공장리뷰,여름에는 너무 덥고 봄,4.0,0,0.2500,"[(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]",NaN


2. OpenAI API 설정 및 기본 함수 정의
이제 OpenAI API를 활용하여 리뷰 텍스트를 분류하는 함수 analyze_sentiment를 정의합니다. API 키는 환경 변수로 설정되어 있으며, 이를 이용해 API 호출을 수행합니다.

# 📘 감정 분석 자동화 로직, 어떻게 돌아갈까요?

이번 섹션에서는 GPT-4o를 활용해서 댓글이 **긍정인지 부정인지 자동으로 판별하는 코드**를 직접 실행해봅니다. 이 코드는 굉장히 중요한 구조를 가지고 있고, 잘 이해하면 여러분도 쉽게 **AI 감정 분석기**를 만들 수 있습니다.

---

## 🔍 먼저 우리가 하려는 일, 한 문장으로 정리하면?

**“리뷰 하나를 GPT에게 보내서 긍정인지 부정인지 판단하고, 그 결과가 정답과 맞는지 비교하자!”**

---

## 🧠 analyze_sentiment 함수: 감정 분석의 핵심 엔진!

이 함수는 GPT 모델에 다음과 같은 역할을 시킵니다.

- "이 리뷰가 긍정적인지 부정적인지 판단해줘"
- "정수로만 대답해줘. 긍정이면 1, 부정이면 -1이야"

즉, 이 함수는 **GPT에게 질문하고 답을 받아오는 인터페이스**입니다.

💡 **포인트 예시:**  
```python
review = "배송도 빠르고 품질도 좋아요!"
→ GPT는 이걸 읽고 → "1" 이라고 응답할 겁니다.


In [ ]:

import openai
import os

# OpenAI API 키 설정 (환경변수에 API 키 저장)


# 감정 분석 함수 정의 (GPT에게 텍스트를 보내 감정을 판단하게 하는 핵심 함수입니다!)
def analyze_sentiment(review, instruction):
    # 💡 포인트: GPT-4o 모델에게 지시문(instruction)과 리뷰(review)를 전달합니다.
    response = openai.chat.completions.create(
        model='gpt-4o',  # 사용할 OpenAI 모델: 최신 고성능 GPT-4o 사용
        messages=[
            {'role': 'system', 'content': instruction},  # GPT에게 역할을 지시하는 시스템 메시지
            {'role': 'user', 'content': review}  # 사용자 역할로 리뷰 내용을 전달합니다.
        ],
        max_tokens=300,  # 최대 응답 길이: 너무 장황하게 답하지 않도록 제한
        temperature=0  # 💡 포인트: 결과의 일관성을 높이기 위해 무작위성은 제거 (0이면 항상 가장 논리적인 응답)
    )
    # GPT가 생성한 응답 중 첫 번째 메시지의 텍스트를 꺼내서 반환합니다.
    return response.choices[0].message.content

# GPT에게 줄 기본 프롬프트 설정
instruction_prompt = '''
주어진 리뷰를 긍정 또는 부정으로 분류하세요. 긍정이면 1, 부정이면 -1로 대답하세요. 정수로만 답하세요
'''



이제 리뷰 데이터를 순회하면서 각 리뷰에 대한 감정 분석을 수행하고, 모델의 예측 결과와 실제 레이블을 비교하여 정확도를 계산합니다.

In [ ]:
# 예측 정확도를 체크하기 위한 카운터 변수 초기화
correct = 0  # 예측이 맞은 경우를 셀 변수
incorrect = 0  # 예측이 틀린 경우를 셀 변수

# 💡 포인트: 아래는 전체 리뷰를 하나씩 꺼내서 GPT에게 감정 분석을 요청하는 루프입니다!
for idx, review, label in zip(reviews['Num'], reviews['text'], reviews['Label']):
    print(f'리뷰 #{idx}: {review}')  # 현재 분석 중인 리뷰 번호와 내용을 출력

    # GPT에게 감정 분석 요청
    prediction = analyze_sentiment(review, instruction_prompt)

    # GPT의 예측 결과와 실제 레이블 비교 출력
    print(f'예측 결과: {prediction} / 실제 레이블: {label}')

    # 예측 결과가 정답과 일치하면 correct++, 아니면 incorrect++
    if prediction == str(label):
        correct += 1
    else:
        incorrect += 1
        print('분류 오류 발생!')  # 예측이 틀린 경우 안내 메시지 출력

    print()  # 가독성을 위한 줄바꿈

# 최종 정확도 계산
accuracy = correct / (correct + incorrect)
print(f'정확도: {accuracy:.2%}')  # 백분율 형식으로 출력

# 성능향상프로젝트

# 모델 변경(gpt4o->gpt 4.1)

In [ ]:
# 감정 분석을 위한 함수 정의
def analyze_sentiment(review, instruction):
    # OpenAI GPT-4 모델을 사용하여 리뷰의 감정을 분석
    # 주어진 리뷰와 지시문을 모델에게 전달하여 응답을 받음
    response = openai.chat.completions.create(
        model='gpt-4.1',  # 사용할 모델은 GPT-4
        messages=[
            {'role': 'system', 'content': instruction},  # 시스템 역할의 지시문
            {'role': 'user', 'content': review}          # 유저가 입력한 리뷰
        ],
        max_tokens=300,  # 응답의 최대 토큰 수 제한 (너무 길어지지 않게)
        temperature=0     # 모델의 일관성을 위해 랜덤성을 줄임 (0이면 가장 확실한 답변)
    )
    # 모델의 응답에서 감정 분석 결과를 추출하여 반환
    return response.choices[0].message.content

# GPT-4에게 리뷰를 긍정/부정으로 분류하도록 지시하는 기본 명령어
instruction_prompt = '''
주어진 리뷰를 긍정 또는 부정으로 분류하세요. 긍정이면 1, 부정이면 -1로 대답하세요. 정수로만 답하세요'''


In [ ]:
# 예측의 정확도를 측정하기 위한 변수 초기화
correct = 0  # 정확히 예측한 리뷰 수
incorrect = 0  # 잘못 예측한 리뷰 수

# 리뷰 데이터에서 각 리뷰와 그에 대응하는 실제 레이블을 하나씩 처리
for idx, review, label in zip(reviews['Num'], reviews['Review'], reviews['Label']):
    print(f'리뷰 #{idx}: {review}')  # 현재 분석 중인 리뷰 번호와 내용을 출력

    # 감정 분석 함수 호출하여 리뷰에 대한 감정 예측 수행
    prediction = analyze_sentiment(review, instruction_prompt)
    print(f'예측 결과: {prediction} / 실제 레이블: {label}')  # 예측 결과와 실제 레이블을 비교하여 출력

    # 예측 결과와 실제 레이블이 일치하는지 확인
    if prediction == str(label):
        correct += 1  # 예측이 맞으면 correct 카운트 증가
    else:
        incorrect += 1  # 예측이 틀리면 incorrect 카운트 증가
        print('분류 오류 발생!')  # 예측 오류가 있음을 출력

    print()  # 가독성을 위해 줄바꿈 추가

# 정확도 계산 (정확하게 예측한 비율 계산)
accuracy = correct / (correct + incorrect)
# 최종 정확도를 백분율로 출력
print(f'정확도: {accuracy:.2%}')

파라미터변경(temperature=0 ->temperature=0.7)

In [ ]:
# 감정 분석을 수행하는 함수 정의
def analyze_sentiment(review, instruction):
    # OpenAI의 GPT-3.5-turbo 모델을 사용하여 감정 분석을 수행
    # 모델에 리뷰 텍스트와 지시문을 전달하여 응답을 받아온다
    response = openai.chat.completions.create(
        model='gpt-4o',  # 사용할 모델 설정 (gpt-4o)
        messages=[
            {'role': 'system', 'content': instruction},  # 시스템 역할의 지시문 (모델에게 작업을 설명)
            {'role': 'user', 'content': review}          # 분석할 리뷰 내용 (사용자가 입력한 텍스트)
        ],
        max_tokens=300,  # 모델이 생성하는 응답의 최대 길이 (300 토큰으로 제한)
        temperature=0.7  # 응답의 무작위성을 설정 (0.7로 약간의 창의성 부여)
    )
    # 모델의 응답에서 감정 분석 결과를 추출하여 반환
    return response.choices[0].message.content

# GPT-3.5-turbo에게 전달할 기본 명령어 (리뷰를 긍정 또는 부정으로 분류하는 프롬프트)
instruction_prompt = '''
주어진 리뷰를 긍정 또는 부정으로 분류하세요. 긍정이면 1, 부정이면 -1로 대답하세요. 정수로만 답하세요'''


In [ ]:
# 예측 정확도를 측정하기 위한 변수 초기화
correct = 0  # 정확한 예측의 개수를 저장하는 변수
incorrect = 0  # 잘못된 예측의 개수를 저장하는 변수

# 리뷰 데이터에서 각 리뷰, 인덱스, 실제 레이블을 순차적으로 가져와서 처리
for idx, review, label in zip(reviews['Num'], reviews['Review'], reviews['Label']):
    print(f'리뷰 #{idx}: {review}')  # 현재 분석 중인 리뷰 번호와 내용을 출력

    # 감정 분석 함수 호출하여 예측 결과를 얻음
    prediction = analyze_sentiment(review, instruction_prompt)
    print(f'예측 결과: {prediction} / 실제 레이블: {label}')  # 모델의 예측 결과와 실제 레이블을 비교 출력

    # 예측 결과가 실제 레이블과 일치하는지 확인
    if prediction == str(label):
        correct += 1  # 예측이 맞으면 correct 카운트 증가
    else:
        incorrect += 1  # 예측이 틀리면 incorrect 카운트 증가
        print('분류 오류 발생!')  # 분류 오류 발생 시 메시지 출력

    print()  # 가독성을 위한 줄바꿈 추가

# 최종 정확도 계산
accuracy = correct / (correct + incorrect)  # 전체 예측 중 정확한 예측의 비율을 계산
print(f'정확도: {accuracy:.2%}')  # 정확도를 백분율로 출력

## 프롬프트 엔지니어링

# CoT기법

아래 코드는 GPT에게 감정 분석을 요청할 때 더 자세한 지침을 주는 방식(CoT, Chain of Thought)을 사용한 예입니다. GPT가 단순히 "좋다/나쁘다"를 판단하는 것이 아니라, 먼저 내용을 꼼꼼히 읽고 감정을 분석한 뒤 숫자로 답하게 유도합니다. 예를 들어, "배송은 빨랐지만 품질이 별로예요"라는 리뷰처럼 혼합된 감정이 있는 경우에도 좀 더 정확한 판단을 할 수 있게 도와줍니다. 이처럼, 프롬프트를 잘 설계하면 GPT의 판단력도 향상된다는 점이 핵심입니다.


In [ ]:
# 감정 분석을 위한 상세한 지침 (프롬프트) 설정
detailed_prompt = '''
Step 1: 리뷰의 내용을 한 줄 한 줄, 한 단어 한 단어 문맥에 맞게 꼼꼼히 확인해서 주어진 리뷰에서 표현된 감성을 분석하세요.
내용이 좋은 내용이면 긍정적이고, 주로 비판하거나 불만을 나타내는 내용을 포함하고 있으면 부정적입니다.
Step 2: 리뷰가 긍정적이면 1 또는 부정적이면 -1로 답하세요. 정수로만 답하세요
'''

# 예측 정확도를 측정하기 위한 변수 초기화
correct = 0  # 정확히 예측한 리뷰 개수 저장
incorrect = 0  # 잘못 예측한 리뷰 개수 저장

# 리뷰 데이터에서 각 리뷰와 그에 해당하는 인덱스, 실제 레이블을 차례대로 가져와 처리
for idx, review, label in zip(reviews['Num'], reviews['Review'], reviews['Label']):
    print(f'리뷰 #{idx}: {review}')  # 현재 처리 중인 리뷰 출력

    # 감정 분석 함수 호출, 리뷰에 대한 예측 결과 받기
    prediction = analyze_sentiment(review, detailed_prompt)
    print(f'예측 결과: {prediction} / 실제 레이블: {label}')  # 예측 결과와 실제 레이블 비교 출력

    # 예측 결과가 실제 레이블과 일치하는지 확인
    if prediction == str(label):
        correct += 1  # 일치하면 올바른 예측으로 카운트 증가
    else:
        incorrect += 1  # 일치하지 않으면 잘못된 예측으로 카운트 증가
        print('분류 오류 발생!')  # 분류 오류 발생 시 메시지 출력

    print()  # 가독성을 위한 줄바꿈

# 최종 정확도 계산
accuracy = correct / (correct + incorrect)  # 정확도 계산 (올바른 예측 / 전체 예측)
print(f'정확도: {accuracy:.2%}')  # 정확도를 백분율로 출력
